# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
G. Green  ->  G. Green  |  ['G. Green']
S. Letchev  ->  S. Letchev  |  ['S. Letchev']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']
S. Li  ->  S. Li  |  ['S. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Mathew  ->  J. Mathew  |  ['J. Mathew']


F. Walter  ->  F. Walter  |  ['F. Walter']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Arxiv has 89 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.14594


/tmp/ipykernel_2155/1211882699.py:51: LatexWarning: 2407.14594 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2407.14606
extracting tarball to tmp_2407.14606...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.14677
extracting tarball to tmp_2407.14677... done.
Retrieving document from  https://arxiv.org/e-print/2407.14856
extracting tarball to tmp_2407.14856... done.
Retrieving document from  https://arxiv.org/e-print/2407.14934


...

 done.
Retrieving document from  https://arxiv.org/e-print/2407.15846


extracting tarball to tmp_2407.15846...

 done.


J. Li  ->  J. Li  |  ['J. Li']
F. Walter  ->  F. Walter  |  ['F. Walter']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 119 bibliographic references in tmp_2407.15846/main.bbl.
syntax error in line 228: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.15572-b31b1b.svg)](https://arxiv.org/abs/2407.15572) | **FEASTS Combined with Interferometry (II): Significantly Changed HI Surface Densities and Even More Inefficient Star Formation in Galaxy Outer Disks**  |
|| J. Wang, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-07-23*|
|*Comments*| *27 pages, the second paper of "FEASTS Combined with Interferometry". Data is available on the page this http URL*|
|**Abstract**|            We update the HI surface density measurements for a subset of 17 THINGS galaxies by dealing with the short-spacing problem of the original VLA HI images. It is the same sample that Bigiel et al. (2010) used to study the relation between HI surface densities and star formation rate surface densities in galaxy outer disks, which are beyond the optical radius r25. For ten galaxies, the update is based on combining original THINGS VLA HI images with HI images taken by the single-dish FAST in the FEASTS program. The median increment of HI surface densities in outer disks is 0.15 to 0.4 dex at a given new HI surface density. Several galaxies change significantly in the shape of radial profiles HI surface densities, and seven galaxies are now more than 1-$\sigma$ below the HI size-mass relation. We update the HI star formation laws in outer disks. The median relation between HI surface densities and star formation rate surface densities based on pixelwise measurements shifts downward by around 0.15 dex because the HI surface density values shift rightward, and the scatter increases significantly. The scatter of the relation, indicating the star forming efficiency, exhibits a much stronger positive correlation with the stellar mass surface density than before. Thus, detecting the previously missed, diffuse HI due to short-spacing problem of the VLA observation is important in revealing the true condition and variation of star formation possibly regulated by stellar feedbacks in localized environment of outer disks.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.15846-b31b1b.svg)](https://arxiv.org/abs/2407.15846) | **ALESS-JWST: Joint (sub-)kiloparsec JWST and ALMA imaging of $z\sim3$ submillimeter galaxies reveals heavily obscured bulge formation events**  |
|| J. A. Hodge, et al. -- incl., <mark>J. Li</mark>, <mark>F. Walter</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-07-23*|
|*Comments*| *26 pages, 10 figures. Submitted to ApJ. Comments welcome*|
|**Abstract**|            We present JWST NIRCam imaging targeting 13 $z\sim3$ infrared-luminous ($L_{\rm IR}\sim5\times10^{12}L_{\odot}$) galaxies from the ALESS survey with uniquely deep, high-resolution (0.08$''$$-$0.16$''$) ALMA 870$\mu$m imaging. The 2.0$-$4.4$\mu$m (observed frame) NIRCam imaging reveals the rest-frame near-infrared stellar emission in these submillimeter-selected galaxies (SMGs) at the same (sub-)kpc resolution as the 870$\mu$m dust continuum. The newly revealed stellar morphologies show striking similarities with the dust continuum morphologies at 870$\mu$m, with the centers and position angles agreeing for most sources, clearly illustrating that the spatial offsets reported previously between the 870$\mu$m and HST morphologies were due to strong differential dust obscuration. The F444W sizes are 78$\pm$21% larger than those measured at 870$\mu$m, in contrast to recent results from hydrodynamical simulations that predict larger 870$\mu$m sizes. We report evidence for significant dust obscuration in F444W for the highest-redshift sources, emphasizing the importance of longer-wavelength MIRI imaging. The majority of the sources show evidence that they are undergoing mergers/interactions, including tidal tails/plumes -- some of which are also detected at 870$\mu$m. We find a clear correlation between NIRCam colors and 870$\mu$m surface brightness on $\sim$1 kpc scales, indicating that the galaxies are primarily red due to dust -- not stellar age -- and we show that the dust structure on $\sim$kpc-scales is broadly similar to that in nearby galaxies. Finally, we find no strong stellar bars in the rest-frame near-infrared, suggesting the extended bar-like features seen at 870$\mu$m are highly obscured and/or gas-dominated structures that are likely early precursors to significant bulge growth.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.14677-b31b1b.svg)](https://arxiv.org/abs/2407.14677) | **Simulation results for Robo-AO-2 using HAPA: a wavefront sensing technique for improving the adaptive optics correction of fainter stars**  |
|| <mark>R. Zhang</mark>, et al. |
|*Appeared on*| *2024-07-23*|
|*Comments*| *11 pages, 4 figures, 8 tables, SPIE Astronomical Telescopes + Instrumentation 2024*|
|**Abstract**|            Direct imaging of exoplanets allows us to measure positions and chemical signatures of exoplanets. Given the limited resources for space observations where the atmosphere is absent, we want to make these measurements from the ground. However, it is difficult from the ground because it requires an adaptive optics system to provide an extremely well corrected wavefront to enable coronographic techniques. Currently only natural guide star AO systems have demonstrated the necessary wavefront correction for direct imaging of exoplanets. However, using a stellar source as the guide star for wavefront sensing limits the number of exoplanet systems we can directly image because it requires a relatively bright V~10 mag star. To increase the number of observable targets, we need to push the limit of natural guide stars to fainter magnitudes with high Strehl ratio correction. We propose to combine laser guide star (LGS) and natural guide star (NGS) wavefront sensing to achieve the high Strehl correction with fainter natural guide stars. We call this approach Hybrid Atmospheric Phase Analysis (HAPA); 'hapa' in Hawaiian means 'half' or 'of mixed ethnic heritage'. The relatively bright LGS is used for higher order correction, whereas the NGS is used for high accuracy lower order correction. We focus on demonstrating this approach using Robo-AO-2 at the UH 2.2m telescope on Maunakea with a UV Rayleigh laser at 355 nm. The laser focuses at 10 km altitude and has an equivalent magnitude of m_U~8. In this report specifically, we present simulated results of HAPA employed at Robo-AO-2, with the LGS system having a single configuration of 16x16 subaperture Shack-Hartmann wavefront sensor and the NGS system having 6 different configurations -- 16x16, 8x8, 5x5, 4x4, 2x2 and 1x1. We also discuss the on-sky experiments we plan to carry out with HAPA at the UH 2.2m telescope.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.15093-b31b1b.svg)](https://arxiv.org/abs/2407.15093) | **Ultraviolet Extinction Sky Survey (UVESS): A mission concept for probing the interstellar medium in the Milky Way and Local Group galaxies**  |
|| <mark>J. Mathew</mark>, et al. |
|*Appeared on*| *2024-07-23*|
|*Comments*| **|
|**Abstract**|            The 2175 Å bump shows considerable variations in its strength, width, and central wavelength when observed along different sightlines in the Milky Way and other galaxies. These variations offer valuable insights into the composition, size distribution, and processing of interstellar dust grains along different sightlines. This paper introduces a mission concept called UVESS (Ultraviolet Extinction Sky Survey) aimed at exploring the composition of the interstellar medium (ISM) within both the Milky Way and nearby Local Group Galaxies by mapping the variation of UV extinction curve slopes and the 2175 Å feature across a majority of the sky to gain insights into the makeup of the ISM. Recent advancements in UV instrumentation and technologies pave the way for the development of high-throughput instruments in compact form factors. In this paper, we outline mission science goals and instrument concept tailored for a small satellite-based platform dedicated to the study of UV extinction.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.15640-b31b1b.svg)](https://arxiv.org/abs/2407.15640) | **Impacts of dark energy on weighing neutrinos after DESI BAO**  |
|| G.-H. Du, P.-J. Wu, T.-N. Li, <mark>X. Zhang</mark> |
|*Appeared on*| *2024-07-23*|
|*Comments*| *10 pages, 4 figures*|
|**Abstract**|            Recently, DESI has released baryon acoustic oscillation (BAO) data, and DES has also published its five-year supernova (SN) data. These observations, combined with cosmic microwave background (CMB) data, support a dynamically evolving dark energy at a high confidence level. When using cosmological observations to weigh neutrinos, the results of weighing neutrinos will be significantly affected by the measurement of dark energy due to the degeneracy between neutrino mass and the dark-energy equation of state. Therefore, we need to understand how the dynamical evolution of dark energy in the current situation will affect the measurement of neutrino mass. In this work, we utilize these latest observations and other additional distance measurements to discuss the mutual influence between neutrinos and dark energy, then calculate the Bayes factor to compare models. We consider three neutrino mass hierarchies including degenerate hierarchy (DH), normal hierarchy (NH), and inverted hierarchy (IH), as well as three dark energy models including $\Lambda \rm CDM$, $w\rm CDM$, and $w_0w_a \rm CDM$ models. Cosmological data combined with the prior of particle physics experiments can provide strong to decisive evidence favoring the $w_0w_a {\rm CDM}+\sum m_\nu$ model with NH. In the $w_0w_a \rm CDM$ model, using the CMB+DESI+DESY5 data, we obtain constraints on the total neutrino mass, $\sum m_\nu<0.171\ \rm eV,\ 0.204\ \rm eV,\ 0.220\ \rm eV$, for DH, NH, and IH, respectively. Furthermore, taking into account the neutrino hierarchy or incorporating additional distance measurements results in a more pronounced deviation from the $\Lambda$CDM model for dark energy. The latter, particularly, exhibits a deviation at a confidence level that surpasses $4\sigma$.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.15768-b31b1b.svg)](https://arxiv.org/abs/2407.15768) | **Early-Time Observations of SN 2023wrk: A Luminous Type Ia Supernova with Significant Unburned Carbon in the Outer Ejecta**  |
|| <mark>J. Liu</mark>, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-07-23*|
|*Comments*| *Accepted for publication in the Astrophysical Journal (27 pages, 14 figures, 7 tables)*|
|**Abstract**|            We present extensive photometric and spectroscopic observations of the nearby Type Ia supernova (SN) 2023wrk at a distance of about 40 Mpc. The earliest detection of this SN can be traced back to a few hours after the explosion. Within the first few days the light curve shows a bump feature, while the B - V color is blue and remains nearly constant. The overall spectral evolution is similar to that of an SN 1991T/SN 1999aa-like SN Ia, while the C II $\lambda6580$ absorption line appears to be unusually strong in the first spectrum taken at $t \approx -$15.4 days after the maximum light. This carbon feature disappears quickly in subsequent evolution but it reappears at around the time of peak brightness. The complex evolution of the carbon line and the possible detection of Ni III absorption around 4700 Å and 5300 Å in the earliest spectra indicate macroscopic mixing of fuel and ash. The strong carbon lines is likely related to collision of SN ejecta with unbound carbon, consistent with the predictions of pulsational delayed-detonation or carbon-rich circumstellar-matter interaction models. Among those carbon-rich SNe Ia with strong C II $\lambda6580$ absorption at very early times, the line-strength ratio of C II to Si II and the B-V color evolution are found to exhibit large diversity, which may be attributed to different properties of unbound carbon and outward-mixing $^{56}$Ni.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.14606-b31b1b.svg)](https://arxiv.org/abs/2407.14606) | **Performance of the Nonlinear Curvature Wavefront Sensor as a Function of Scintillation Strength**  |
|| <mark>S. Letchev</mark>, et al. |
|*Appeared on*| *2024-07-23*|
|*Comments*| *6 pages, 3 figures, SPIE Astronomical Telescopes + Instrumentation 2024*|
|**Abstract**|            Local amplitude aberrations caused by scintillation can impact the reconstruction process of a wavefront sensor (WFS) by inducing a spatially non-uniform intensity at the pupil plane. This effect is especially relevant for the commonly-used Shack-Hartmann WFS (SHWFS), which can lose slope information for portions of the beam where the signal is faint, leading to reduced reconstruction performance and eventually total failure as the level of scintillation increases. An alternative WFS is needed for such conditions. The nonlinear curvature wavefront sensor (nlCWFS) has been shown to achieve better sensitivity compared to the SHWFS under low light levels. Additionally, the nlCWFS has demonstrated the ability to maintain its sensitivity in the presence of scintillation, using amplitude aberrations to help inform the reconstruction process, rather than hinder. Experiments to date have thus far only shown reconstruction results for a single scintillation value. Building upon previous simulations and laboratory experiments, we have built a testbed to quantify the effects of varying scintillation strength on the wavefront reconstruction performance of the nlCWFS compared to an equivalent SHWFS. In this paper, we present results showing the difference in performance between the nlCWFS and SHWFS as a function of relative flux and scintillation strength.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.14594-b31b1b.svg)](https://arxiv.org/abs/2407.14594) | **Unveiling the Milky Way dust extinction curve in 3D**  |
|| <mark>X. Zhang</mark>, <mark>G. Green</mark> |
|*Appeared on*| *2024-07-23*|
|*Comments*| *Under review, submitted on 1 March*|
|**Abstract**|            Interstellar dust is a major foreground contaminant for many observations and a key component in the chemistry of the interstellar medium, yet its properties remain highly uncertain. Using low-resolution spectra, we accurately measure the extinction curve - a diagnostic of the grain properties - for 130 million stars, orders of magnitude more than previously available, allowing us to map its variation in the Milky Way and Magellanic Clouds in 3D in unprecedented detail. We find evidence that accretion is the dominant mechanism of grain growth in moderately dense regions, with coagulation dominating at higher densities. Moreover, we find that the extinction curve flattens in star-forming regions, possibly caused by cycling of large grains formed in molecular clouds, or by preferential destruction of small grains by supernova shocks.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.15572/./pdisk2d2_dif_NHI_ref.png', 'tmp_2407.15572/./628_NHI_prof.png', 'tmp_2407.15572/./925_NHI_prof.png', 'tmp_2407.15572/./2841_NHI_prof.png', 'tmp_2407.15572/./3198_NHI_prof.png', 'tmp_2407.15572/./2903_NHI_prof.png', 'tmp_2407.15572/./3521_NHI_prof.png', 'tmp_2407.15572/./5055_NHI_prof.png', 'tmp_2407.15572/./5194_NHI_prof.png']
copying  tmp_2407.15572/./pdisk2d2_dif_NHI_ref.png to _build/html/
copying  tmp_2407.15572/./628_NHI_prof.png to _build/html/
copying  tmp_2407.15572/./925_NHI_prof.png to _build/html/
copying  tmp_2407.15572/./2841_NHI_prof.png to _build/html/
copying  tmp_2407.15572/./3198_NHI_prof.png to _build/html/
copying  tmp_2407.15572/./2903_NHI_prof.png to _build/html/
copying  tmp_2407.15572/./3521_NHI_prof.png to _build/html/
copying  tmp_2407.15572/./5055_NHI_prof.png to _build/html/
copying  tmp_2407.15572/./5194_NHI_prof.png to _build/html/
exported in  _build/html/2407.15572.md
    + _build/html/tmp_2407.15572/./pdisk2d2_dif_NHI_re

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\hi}{{\rm H}{\textsc i}}$
$\newcommand{\angstrom}{\text{\normalfontÅ}}$
$\newcommand{◦ee}{\ensuremath{\text{\textdegree}}}$
$\newcommand{\jyb}{\rm{Jy~beam^{-1} }}$
$\newcommand{\jykms}{\rm{Jy~km~s^{-1} }}$
$\newcommand{\jybkms}{\rm{Jy~beam^{-1}~km~s^{-1} }}$
$\newcommand{\mjybi}{\rm{mJy~beam_{i}^{-1} }}$
$\newcommand{\mjyb}{\rm{mJy~beam^{-1} }}$
$\newcommand{\mjybs}{\rm{mJy~beam_{s}^{-1} }}$
$\newcommand{\jybs}{\rm{Jy~beam_{s}^{-1} }}$
$\newcommand{\mjybkms}{\rm{mJy~beam^{-1}~km~s^{-1} }}$
$\newcommand{\kms}{\rm{km~s^{-1} }}$
$\newcommand{\cmsq}{\rm{ cm^{-2} } }$
$\newcommand{\NHI}{N_{\rm HI}}$
$\newcommand{\Msun}{\rm{M_{\odot}}}$
$\newcommand{\Msunyr}{\rm{M_{\odot}} ~{\rm yr}^{-1} }$
$\newcommand{\Msunyrkpcsq}{\rm{M_{\odot}} ~{\rm yr}^{-1}~{\rm kpc}^{-2} }$
$\newcommand{\Msunpcsq}{\rm{M_{\odot}} ~{\rm pc}^{-2}}$
$\newcommand{\SHI}{\Sigma_{\rm HI}}$
$\newcommand{\Ssfr}{\Sigma_{\rm SFR}}$
$\newcommand{\Sst}{\Sigma_*}$</div>



<div id="title">

#  FEASTS Combined with Interferometry (II):Significantly Changed HI Surface Densities and Even More Inefficient Star Formation in Galaxy Outer Disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.15572-b31b1b.svg)](https://arxiv.org/abs/2407.15572)<mark>Appeared on: 2024-07-23</mark> -  _27 pages, the second paper of "FEASTS Combined with Interferometry". Data is available on the page this http URL_

</div>
<div id="authors">

J. W. (王菁), et al. -- incl., <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We update the $\hi$ surface density measurements for a subset of 17 THINGS galaxies by dealing with the short-spacing problem of the original VLA $\hi$ images.It is the same sample that [Bigiel, Leroy and Walter (2010)]() used to study the relation between $\hi$ surface densities and star formation rate surface densities in galaxy outer disks, which are beyond the optical radius $r_{25}$ .For ten galaxies, the update is based on combining original THINGS VLA $\hi$ images with $\hi$ images taken by the single-dish FAST in the FEASTS program.The median increment of $\hi$ surface densities in outer disks is 0.15 to 0.4 dex at a given new $\hi$ surface density.Several galaxies change significantly in the shape of radial profiles $\hi$ surface densities, and seven galaxies are now more than 1- $\sigma$ below the $\hi$ size-mass relation.We update the $\hi$ star formation laws in outer disks.The median relation between $\hi$ surface densities and star formation rate surface densities based on pixelwise measurements shifts downward by around 0.15 dex because the $\hi$ surface density values shift rightward, and the scatter increases significantly.The scatter of the relation, indicating the star forming efficiency, exhibits a much stronger positive correlation with the stellar mass surface density than before.Thus, detecting the previously missed, diffuse $\hi$ due to short-spacing problem of the VLA observation is important in revealing the true condition and variation of star formation possibly regulated by stellar feedbacks in localized environment of outer disks.

</div>

<div id="div_fig1">

<img src="tmp_2407.15572/./pdisk2d2_dif_NHI_ref.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
The recovering capabilities of pixelwise $\SHI$ of different types of $\hi$ images for the $\kappa=2.2$\& SNR$=2.8$ mock dataset.
The simulated disk based on which the mock images are taken has a power spectral index $\kappa=2.2$, and the VLA data has a median SNR$=2.8$.
Different colors show different types of mock $\hi$ images from which the measurements are made.
For each mock $\hi$ image type, the logarithm difference of $\SHI$ measured from mock images from the true values are plotted as a function of the true $\SHI$.
The solid, thick curves show the median distribution of $\SHI$ difference as a function of true $\SHI$, and the dotted curves the scatters.
The scatters are measured as the 25 and 75 percentiles.
  (*fig:pixNHI_mock_ref*)

</div>
<div id="div_fig2">

<img src="tmp_2407.15572/./628_NHI_prof.png" alt="Fig5.1" width="25%"/><img src="tmp_2407.15572/./925_NHI_prof.png" alt="Fig5.2" width="25%"/><img src="tmp_2407.15572/./2841_NHI_prof.png" alt="Fig5.3" width="25%"/><img src="tmp_2407.15572/./3198_NHI_prof.png" alt="Fig5.4" width="25%"/>

**Figure 5. -**  Comparing radial profiles of $\SHI$ measured from different images for galaxies in the FEASTS subset.
Each row is for a galaxy, with the left panel showing the radial profiles and the right panel the profile difference with respect to the $I_\text{W08}$ measurements.
For each galaxy, the comparison involves two sets of THINGS-only measurements, $I_\text{W08}$(olive) and $I_\text{W24}$(green), based on the W08 and W24 residual-rescaled data cubes respectively.
It also involves two sets of combined images, $C_\text{W08}$(cyan) and $C_\text{W24}$(blue), using the W08 residual-rescaled image and the W24 convolved model to combined with the FEASTS image respectively.
It also includes profiles directly measured from the FEASTS images (red).
The pink and brown dotted vertical lines mark the positions of $R_{\rm HI}$(derived from $C_\text{W24}$ profile) and $r_{25}$ respectively.
To be continued.
 (*fig:prof1*)

</div>
<div id="div_fig3">

<img src="tmp_2407.15572/./2903_NHI_prof.png" alt="Fig6.1" width="25%"/><img src="tmp_2407.15572/./3521_NHI_prof.png" alt="Fig6.2" width="25%"/><img src="tmp_2407.15572/./5055_NHI_prof.png" alt="Fig6.3" width="25%"/><img src="tmp_2407.15572/./5194_NHI_prof.png" alt="Fig6.4" width="25%"/>

**Figure 6. -** Comparing radial profiles of $\SHI$ measured from different images for galaxies in the FEASTS subset.
Continued.   (*fig:prof1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.15572"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\baselinestretch}{1.1}$
$\newcommand{\baselinestretch}{1.0}$</div>



<div id="title">

# ALESS-JWST: Joint (sub-)kiloparsec JWST and ALMA imaging of $z\sim3$ submillimeter galaxies reveals heavily obscured bulge formation events

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.15846-b31b1b.svg)](https://arxiv.org/abs/2407.15846)<mark>Appeared on: 2024-07-23</mark> -  _26 pages, 10 figures. Submitted to ApJ. Comments welcome_

</div>
<div id="authors">

J. A. Hodge, et al. -- incl., <mark>J. Li</mark>, <mark>F. Walter</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** We present JWST NIRCam imaging targeting 13 $z$ $\sim$ 3 infrared-luminous ( $L_{\rm IR}\sim5\times10^{12}L_{\odot}$ ) galaxies from the ALESS survey with uniquely deep, high-resolution (0.08 $"$ --0.16 $"$ ) ALMA 870 $\mu$ m imaging. The 2.0-4.4 $\mu$ m (observed frame) NIRCam imaging reveals the rest-frame near-infrared stellar emission in these submillimeter-selected galaxies (SMGs) at the same (sub-)kpc resolution as the 870 $\mu$ m dust continuum. The newly revealed stellar morphologies show striking similarities with the dust continuum morphologies at 870 $\mu$ m, with the centers and position angles agreeing for most sources, clearly illustrating that the spatial offsets reported previously between the 870 $\mu$ m and HST morphologies were due to strong differential dust obscuration. The F444W sizes are 78 $\pm$ 21 \% larger than those measured at 870 $\mu$ m, in contrast to recent results from hydrodynamical simulations that predict larger 870 $\mu$ m sizes. We report evidence for significant dust obscuration in F444W for the highest-redshift sources, emphasizing the importance of longer-wavelength MIRI imaging. The majority of the sources show evidence that they are undergoing mergers/interactions, including tidal tails/plumes---some of which are also detected at 870 $\mu$ m. We find a clear correlation between NIRCam colors and 870 $\mu$ m surface brightness on $\sim$ 1 kpc scales, indicating that the galaxies are primarily red due to dust---not stellar age---and we show that the dust structure on $\sim$ kpc-scales is broadly similar to that in nearby galaxies. Finally, we find no strong stellar bars in the rest-frame near-infrared, suggesting the extended bar-like features seen at 870 $\mu$ m are highly obscured and/or gas-dominated structures that are likely early precursors to significant bulge growth.

</div>

<div id="div_fig1">

<img src="tmp_2407.15846/./CC-diagram.png" alt="Fig8" width="100%"/>

**Figure 8. -** NIRCam color-color diagrams showing F200W--F356W versus F356W--F444W colors for individual 60$\times$60 mas regions within each galaxy with $>$5$\sigma$ detections in all three NIRCam filters. The data points are color-coded by their corresponding 870$\mu$m flux surface density if the aperture has at least a 1$\sigma$ detection in the ALMA 870$\mu$m map; otherwise the data points are colored black.
Vectors indicating the predicted impact on this color space of varying $A_{\rm V}$ are shown for comparison and were obtained with {\sc Magphys}\citep[][see text for details]{daCunha_2008, daCunha_2015}.
 The vectors are plotted with a shift 1 dex to the left for clarity. The majority of the sources show a correlation between redder NIRCam colors and 870$\mu$m surface brightness.
 (*fig:color-color*)

</div>
<div id="div_fig2">

<img src="tmp_2407.15846/./ALESS_JWSTgalfit_all_page1.png" alt="Fig5.1" width="50%"/><img src="tmp_2407.15846/./ALESS_JWSTgalfit_all_page2.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** {\sc Galfit} modeling of the NIRCam F444W images for the
13 targeted SMGs. For each source, the columns show the NIRCam F444W image (1st column), the best-fit Sérsic profile model (2nd column),
 the residuals (3rd column), as well as a comparison of those residuals to the ALMA 870$\mu$m contours overlaid in red (4th column).
The first and second column are scaled with a linear stretch between 0.25\% and 99\%, with contours showing 20\%, 40\%, 60\%, and 80\% of the peak intensity of the data (i.e., column 1). The third and fourth columns are shown with a linear stretch and the peak intensity scaled down by a factor of 5 to highlight the discrepancies. We note there is no qualitative difference to the residuals when the fits with freely varying $n$ values are shown instead.
870$\mu$m contours start at $\pm$2$\sigma$ and increase in powers of $\sqrt{2}$, where $\sigma=$19$\mu$Jy. Panels are 2$"\times$2$"$, or $\sim$15 kpc at $z\sim3$. See Section \ref{sec:galfit} for details.  The cyan cross overlaid on the third column for ALESS 17.1 shows the position of the known X-ray AGN
\citep[corrected for the median offset between the X-ray catalog and Gaia DR1;][]{Luo_2017}, indicating it is associated with the optically bright companion rather than the SMG.
 (*fig:JWSTgalfit*)

</div>
<div id="div_fig3">

<img src="tmp_2407.15846/./ALESS_JWST_allNRC_all_page1.png" alt="Fig3.1" width="50%"/><img src="tmp_2407.15846/./ALESS_JWST_allNRC_all_page2.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** JWST NIRCam cutouts (3$"\times$3$"$, or $\sim$25$\times$25 kpc at $z\sim3$)
of the 13 sources. For each galaxy, the columns show (from left): F200W, F356W, F444W, and the corresponding RGB image. Individual filters are scaled with a linear stretch between 0.25\% and 99.5\%, except for ALESS 17.1, where a maximum of 95.0\% is used to highlight the faint red submillimeter galaxy to the West and the brighter companion to the East. Most of the targeted SMGs are heavily dust-attenuated even with NIRCam's 2$\mu$m filter but are significantly less obscured above 3.5$\mu$m ($\simeq$1$\mu$m rest-frame). Some examples of apparent tidal tails are highlighted with boxes in the filter where they are most visible, and the panels are labeled with redshift and visual classification (Disk, Indeterminate, or Merger; See Section \ref{sec:classification}.)  (*fig:NRCfilters*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.15846"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

140  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
